In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [35]:
orders = pd.read_csv('orders_ordered.csv')
drivers = pd.read_csv('drivers.csv')

In [3]:
def evaluate(r_history):
    idlings = np.array(list(map(lambda history: history.distance_to_pickup, r_history)))
    print("Mean idlings:\t\t", idlings.mean())
    print("Standard deviation:\t", idlings.std())
    print("Minimum idlings:\t", idlings.min())
    print("Maximum idlings:\t", idlings.max())
    print("25th percentile:\t", np.percentile(idlings, 25))
    print("Median idlings:\t\t", np.median(idlings))
    print("75th percentile:\t", np.percentile(idlings, 75))
    return idlings

In [62]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return np.sqrt(np.square(lat1 - lat2) + np.square(lon1 - lon2))

In [5]:
from simulation_engine import SimulationEngine

In [36]:
se = SimulationEngine(
    drivers_df=drivers, 
    orders_df=orders[:50000],
    simulation_step_delta=timedelta(minutes=30),
    available_orders_expire_time=timedelta(hours=1)
)

In [59]:
se.set_simulation_step_delta(timedelta(minutes=10))

# Baseline random


In [60]:
for time in se:
    orders = se.get_available_orders()
    drivers = se.get_available_drivers()

    max_orders = len(orders)
    for i, driver in enumerate(drivers):
        if i < max_orders:
            ride_distance = calculate_distance(orders[i].pickup_coord.lat, orders[i].pickup_coord.lon, orders[i].dropoff_coord.lat, orders[i].dropoff_coord.lon)
            se.set_ride(
                driver_id=driver.id,
                order_id=orders[i].id,
                distance_to_pickup=calculate_distance(orders[i].pickup_coord.lat, orders[i].pickup_coord.lon, driver.current_coord.lat, driver.current_coord.lon),
                distance_from_pickup_to_dropoff=ride_distance,
                end_time=time+timedelta(seconds=ride_distance/5.827987745770005e-05),
            )
        else:
            break

In [51]:
r_history = se.get_rides_history()
evaluate(r_history)
print("\nnum of expired:\t\t", len(se.get_expired_order()))

Mean idlings:		 0.11397798286232819
Standard deviation:	 0.078501246130149
Minimum idlings:	 0.0
Maximum idlings:	 2.028886548725684
25th percentile:	 0.06265736748205025
Median idlings:		 0.10149155826809267
75th percentile:	 0.14601189362545935

num of expired:		 0


# Baseline nearest driver

In [53]:
for time in se:
    print(f'Simulation time {time}')
    orders = se.get_available_orders()
    drivers = se.get_available_drivers()
    
    max_drivers = len(drivers)
    for i, order in enumerate(orders):
        if i < max_drivers:
            drivers = se.get_available_drivers()
            drivers_coords = np.array(list(map(lambda driver: np.array([driver.current_coord.lat, driver.current_coord.lon]), drivers)))
            pickup_coords = np.array([order.pickup_coord.lat, order.pickup_coord.lon])
            
            driver = drivers[np.sum(np.square(drivers_coords - pickup_coords), axis=1).argmin()]
            ride_distance = calculate_distance(order.pickup_coord.lat, order.pickup_coord.lon, order.dropoff_coord.lat, order.dropoff_coord.lon)

            se.set_ride(
                driver_id=driver.id,
                order_id=order.id,
                distance_to_pickup=calculate_distance(order.pickup_coord.lat, order.pickup_coord.lon, driver.current_coord.lat, driver.current_coord.lon),
                distance_from_pickup_to_dropoff=ride_distance,
                end_time=time+timedelta(seconds=ride_distance/5.827987745770005e-05),
            )
        else:
            break

Simulation time 2020-05-08 00:00:00
Simulation time 2020-05-08 01:00:00
Simulation time 2020-05-08 02:00:00
Simulation time 2020-05-08 03:00:00
Simulation time 2020-05-08 04:00:00
Simulation time 2020-05-08 05:00:00
Simulation time 2020-05-08 06:00:00
Simulation time 2020-05-08 07:00:00
Simulation time 2020-05-08 08:00:00
Simulation time 2020-05-08 09:00:00
Simulation time 2020-05-08 10:00:00
Simulation time 2020-05-08 11:00:00
Simulation time 2020-05-08 12:00:00
Simulation time 2020-05-08 13:00:00
Simulation time 2020-05-08 14:00:00
Simulation time 2020-05-08 15:00:00
Simulation time 2020-05-08 16:00:00
Simulation time 2020-05-08 17:00:00
Simulation time 2020-05-08 18:00:00
Simulation time 2020-05-08 19:00:00
Simulation time 2020-05-08 20:00:00
Simulation time 2020-05-08 21:00:00
Simulation time 2020-05-08 22:00:00
Simulation time 2020-05-08 23:00:00
Simulation time 2020-05-09 00:00:00
Simulation time 2020-05-09 01:00:00
Simulation time 2020-05-09 02:00:00
Simulation time 2020-05-09 0

KeyboardInterrupt: 

In [61]:
r_history = se.get_rides_history()
evaluate(r_history)
print("\nnum of expired:\t\t", len(se.get_expired_order()))

Mean idlings:		 0.0799868144891374
Standard deviation:	 0.05525696426331508
Minimum idlings:	 0.0
Maximum idlings:	 1.4892714180363518
25th percentile:	 0.043790597531896125
Median idlings:		 0.07103660755912251
75th percentile:	 0.10260012232653863

num of expired:		 0


# new ds

In [6]:
se1 = SimulationEngine(
    drivers_df=drivers, 
    orders_df=orders[:50000],
    simulation_step_delta=timedelta(minutes=10),
    available_orders_expire_time=timedelta(hours=1)
)

/home/sergey/Projects/Uklon-MVP/simulation_engine.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._orders_df['pickupTime_converted'] = pd.to_datetime(self._orders_df['pickupTime'])


# Baseline random from 5-nearest driver

In [27]:
for time in se1:
    print(f'Simulation time {time}')
    orders = se1.get_available_orders()
    drivers = se1.get_available_drivers()
    
    max_drivers = len(drivers)
    for i, order in enumerate(orders):
        if i < max_drivers:
            drivers = se1.get_available_drivers()
            drivers_coords = np.array(list(map(lambda driver: np.array([driver.current_coord.lat, driver.current_coord.lon]), drivers)))
            pickup_coords = np.array([order.pickup_coord.lat, order.pickup_coord.lon])
            
            distance = np.sum(np.square(drivers_coords - pickup_coords), axis=1)
            indxs = np.argpartition(distance, -5)[-5:]
            ind = np.random.choice(indxs)
            driver = drivers[ind]

            ride_distance = calculate_distance(order.pickup_coord.lat, order.pickup_coord.lon, order.dropoff_coord.lat, order.dropoff_coord.lon)

            se1.set_ride(
                driver_id=driver.id,
                order_id=order.id,
                distance_to_pickup=calculate_distance(order.pickup_coord.lat, order.pickup_coord.lon, driver.current_coord.lat, driver.current_coord.lon),
                distance_from_pickup_to_dropoff=ride_distance,
                end_time=time+timedelta(seconds=ride_distance/5.827987745770005e-05),
            )
        else:
            break

Simulation time 2020-05-08 00:00:00
Simulation time 2020-05-08 00:10:00
Simulation time 2020-05-08 00:20:00
Simulation time 2020-05-08 00:30:00
Simulation time 2020-05-08 00:40:00
Simulation time 2020-05-08 00:50:00
Simulation time 2020-05-08 01:00:00
Simulation time 2020-05-08 01:10:00
Simulation time 2020-05-08 01:20:00
Simulation time 2020-05-08 01:30:00
Simulation time 2020-05-08 01:40:00
Simulation time 2020-05-08 01:50:00
Simulation time 2020-05-08 02:00:00
Simulation time 2020-05-08 02:10:00
Simulation time 2020-05-08 02:20:00
Simulation time 2020-05-08 02:30:00
Simulation time 2020-05-08 02:40:00
Simulation time 2020-05-08 02:50:00
Simulation time 2020-05-08 03:00:00
Simulation time 2020-05-08 03:10:00
Simulation time 2020-05-08 03:20:00
Simulation time 2020-05-08 03:30:00
Simulation time 2020-05-08 03:40:00
Simulation time 2020-05-08 03:50:00
Simulation time 2020-05-08 04:00:00
Simulation time 2020-05-08 04:10:00
Simulation time 2020-05-08 04:20:00
Simulation time 2020-05-08 0

In [54]:
r_history = se1.get_rides_history()
evaluate(r_history)
print("\nnum of expired:\t\t", len(se1.get_expired_order()))

Mean idlings:		 0.07722627942360885
Standard deviation:	 0.03710693263958038
Minimum idlings:	 0.01380076809456939
Maximum idlings:	 1.030916147172021
25th percentile:	 0.05496109533113717
Median idlings:		 0.07068746246203697
75th percentile:	 0.08921465462579632

num of expired:		 0


# Baseline random from 5-nearest driver include nearest ended

In [7]:
for time in se1:
    print(f'Simulation time {time}')
    orders = se1.get_available_orders()
    drivers = se1.get_available_drivers()
    
    max_drivers = len(drivers)
    for i, order in enumerate(orders):
        semi = se1.get_semi_available_drivers()
        drivers = se1.get_available_drivers()
                
        drivers = semi + drivers
        
        if len(drivers) == 0:
            break

        drivers_coords = np.array(list(map(lambda driver: np.array([driver.current_coord.lat, driver.current_coord.lon]), drivers)))
        pickup_coords = np.array([order.pickup_coord.lat, order.pickup_coord.lon])

        distance = np.sum(np.square(drivers_coords - pickup_coords), axis=1)
        indxs = np.argpartition(distance, -5)[-5:]
        ind = np.random.choice(indxs)
        driver = drivers[ind]

        ride_distance = calculate_distance(order.pickup_coord.lat, order.pickup_coord.lon, order.dropoff_coord.lat, order.dropoff_coord.lon)

        se1.set_ride(
            driver_id=driver.id,
            order_id=order.id,
            distance_to_pickup=calculate_distance(order.pickup_coord.lat, order.pickup_coord.lon, driver.current_coord.lat, driver.current_coord.lon),
            distance_from_pickup_to_dropoff=ride_distance,
            end_time=time+timedelta(seconds=ride_distance/5.827987745770005e-05),
        )

Simulation time 2020-05-08 00:00:00
Simulation time 2020-05-08 00:10:00
Simulation time 2020-05-08 00:20:00
Simulation time 2020-05-08 00:30:00
Simulation time 2020-05-08 00:40:00
Simulation time 2020-05-08 00:50:00
Simulation time 2020-05-08 01:00:00
Simulation time 2020-05-08 01:10:00
Simulation time 2020-05-08 01:20:00
Simulation time 2020-05-08 01:30:00
Simulation time 2020-05-08 01:40:00
Simulation time 2020-05-08 01:50:00
Simulation time 2020-05-08 02:00:00
Simulation time 2020-05-08 02:10:00
Simulation time 2020-05-08 02:20:00
Simulation time 2020-05-08 02:30:00
Simulation time 2020-05-08 02:40:00
Simulation time 2020-05-08 02:50:00
Simulation time 2020-05-08 03:00:00
Simulation time 2020-05-08 03:10:00
Simulation time 2020-05-08 03:20:00
Simulation time 2020-05-08 03:30:00
Simulation time 2020-05-08 03:40:00
Simulation time 2020-05-08 03:50:00
Simulation time 2020-05-08 04:00:00
Simulation time 2020-05-08 04:10:00
Simulation time 2020-05-08 04:20:00
Simulation time 2020-05-08 0

In [8]:
r_history = se1.get_rides_history()
evaluate(r_history)
print("\nnum of expired:\t\t", len(se1.get_expired_order()))

Mean idlings:		 0.07722627942360885
Standard deviation:	 0.03710693263958038
Minimum idlings:	 0.01380076809456939
Maximum idlings:	 1.030916147172021
25th percentile:	 0.05496109533113717
Median idlings:		 0.07068746246203697
75th percentile:	 0.08921465462579632

num of expired:		 0
